In [1]:
import pandas as pd
import joblib
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
data_raw = '../raw/'

In [3]:
games = pd.read_csv(data_raw + 'games.csv')

In [4]:
clubs = pd.read_csv(data_raw + 'clubs.csv')

In [5]:
club_games = pd.read_csv(data_raw + 'club_games.csv')

In [6]:
competitions = pd.read_csv(data_raw + 'competitions.csv')

In [7]:
players = pd.read_csv(data_raw + 'players.csv')

# Визначення структури датасету

In [8]:
df = games.merge(
    competitions,
    left_on='competition_id',
    right_on='competition_id',
)

In [9]:
clubs.head()

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url
0,105,sv-darmstadt-98,SV Darmstadt 98,L1,NaN,27,25.6,13,48.1,1,Merck-Stadion am Böllenfalltor,17810,+€3.05m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/sv-darmstadt-9...
1,11127,ural-ekaterinburg,Ural Yekaterinburg,RU1,NaN,30,26.5,11,36.7,3,Yekaterinburg Arena,23000,+€880k,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/ural-ekaterinb...
2,114,besiktas-istanbul,Beşiktaş Jimnastik Kulübü,TR1,NaN,30,26.6,15,50.0,8,Beşiktaş Park,42445,€-25.26m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/besiktas-istan...
3,12,as-rom,Associazione Sportiva Roma,IT1,NaN,26,26.3,18,69.2,17,Olimpico di Roma,70634,€-76.90m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/as-rom/startse...
4,148,tottenham-hotspur,Tottenham Hotspur Football Club,GB1,NaN,30,25.5,21,70.0,18,Tottenham Hotspur Stadium,62850,€-120.05m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/tottenham-hots...


In [10]:
club_market_values = players.groupby('current_club_id').agg(
    total_market_value=('market_value_in_eur', 'sum'),
    total_market_value_max=('highest_market_value_in_eur', 'sum')
).reset_index()

clubs = clubs.drop(columns=['total_market_value'])

clubs = clubs.merge(
    club_market_values,
    left_on='club_id',
    right_on='current_club_id',
    how='left'
).drop(columns=['current_club_id'])

In [11]:
clubs = clubs.merge(
    club_games.groupby('club_id').agg(total_is_win=('is_win', 'sum')).reset_index(),
    on='club_id',
    how='left'
)

In [12]:
clubs.head()

,club_id,club_code,name,domestic_competition_id,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url,total_market_value,total_market_value_max,total_is_win
0,105,sv-darmstadt-98,SV Darmstadt 98,L1,27,25.6,13,48.1,1,Merck-Stadion am Böllenfalltor,17810,+€3.05m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/sv-darmstadt-9...,40075000.0,1.046750e+08,31
1,11127,ural-ekaterinburg,Ural Yekaterinburg,RU1,30,26.5,11,36.7,3,Yekaterinburg Arena,23000,+€880k,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/ural-ekaterinb...,36675000.0,1.651500e+08,128
2,114,besiktas-istanbul,Beşiktaş Jimnastik Kulübü,TR1,30,26.6,15,50.0,8,Beşiktaş Park,42445,€-25.26m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/besiktas-istan...,146880000.0,5.749000e+08,289
3,12,as-rom,Associazione Sportiva Roma,IT1,26,26.3,18,69.2,17,Olimpico di Roma,70634,€-76.90m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/as-rom/startse...,387920000.0,1.131975e+09,341
4,148,tottenham-hotspur,Tottenham Hotspur Football Club,GB1,30,25.5,21,70.0,18,Tottenham Hotspur Stadium,62850,€-120.05m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/tottenham-hots...,884450000.0,1.349600e+09,345


In [13]:
clubs.to_csv('../processed/clubs_mod.csv', index=False)

In [14]:
clubs_renamed = clubs.rename(
    columns={col: f"{col}_home" for col in clubs.columns if col != 'club_id'}
)

df = df.merge(
    clubs_renamed,
    left_on='home_club_id',
    right_on='club_id'
)

In [15]:
clubs_renamed = clubs.rename(
    columns={col: f"{col}_away" for col in clubs.columns if col != 'club_id'}
)

df = df.merge(
    clubs_renamed,
    left_on='away_club_id',
    right_on='club_id'
)

del clubs_renamed

In [16]:
print(df.shape)
df.info()

(58486, 71)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58486 entries, 0 to 58485
Data columns (total 71 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   game_id                       58486 non-null  int64  
 1   competition_id                58486 non-null  object 
 2   season                        58486 non-null  int64  
 3   round                         58486 non-null  object 
 4   date                          58486 non-null  object 
 5   home_club_id                  58486 non-null  float64
 6   away_club_id                  58486 non-null  float64
 7   home_club_goals               58486 non-null  float64
 8   away_club_goals               58486 non-null  float64
 9   home_club_position            51559 non-null  float64
 10  away_club_position            51559 non-null  float64
 11  home_club_manager_name        58308 non-null  object 
 12  away_club_manager_name        58308 non-null  ob

In [17]:
null_counts = df.isnull().sum()
null_counts = null_counts[null_counts > 0]
null_counts = null_counts[null_counts == len(df)]
null_counts

coach_name_home    58486
coach_name_away    58486
dtype: int64

In [18]:
df = df.drop(columns=['coach_name_home'])
df = df.drop(columns=['coach_name_away'])

In [19]:
print(df.shape)
df.info()

(58486, 69)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58486 entries, 0 to 58485
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   game_id                       58486 non-null  int64  
 1   competition_id                58486 non-null  object 
 2   season                        58486 non-null  int64  
 3   round                         58486 non-null  object 
 4   date                          58486 non-null  object 
 5   home_club_id                  58486 non-null  float64
 6   away_club_id                  58486 non-null  float64
 7   home_club_goals               58486 non-null  float64
 8   away_club_goals               58486 non-null  float64
 9   home_club_position            51559 non-null  float64
 10  away_club_position            51559 non-null  float64
 11  home_club_manager_name        58308 non-null  object 
 12  away_club_manager_name        58308 non-null  ob

In [20]:
df.head(10)

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,away_club_position,home_club_manager_name,away_club_manager_name,stadium,attendance,referee,url_x,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type,competition_code,name,sub_type,type,country_id,country_name,domestic_league_code,confederation,url_y,is_major_national_league,club_id_x,club_code_home,name_home,domestic_competition_id_home,squad_size_home,average_age_home,foreigners_number_home,foreigners_percentage_home,national_team_players_home,stadium_name_home,stadium_seats_home,net_transfer_record_home,last_season_home,filename_home,url_home,total_market_value_home,total_market_value_max_home,total_is_win_home,club_id_y,club_code_away,name_away,domestic_competition_id_away,squad_size_away,average_age_away,foreigners_number_away,foreigners_percentage_away,national_team_players_away,stadium_name_away,stadium_seats_away,net_transfer_record_away,last_season_away,filename_away,url_away,total_market_value_away,total_market_value_max_away,total_is_win_away
0,2321027,L1,2013,1. Matchday,2013-08-11,33.0,41.0,3.0,3.0,8.0,9.0,Jens Keller,Thorsten Fink,Veltins-Arena,61973.0,Manuel Gräfe,https://www.transfermarkt.co.uk/fc-schalke-04_...,4-2-3-1,4-2-3-1,FC Schalke 04,Hamburger SV,3:3,domestic_league,bundesliga,bundesliga,first_tier,domestic_league,40,Germany,L1,europa,https://www.transfermarkt.co.uk/bundesliga/sta...,True,33,fc-schalke-04,FC Schalke 04,L1,27,26.4,12,44.4,4,Veltins-Arena,62271,+€10.30m,2022,../data/raw/transfermarkt-scraper/2022/clubs.j...,https://www.transfermarkt.co.uk/fc-schalke-04/...,26375000.0,190875000.0,167,41,hamburger-sv,Hamburger SV,L1,30,23.6,15,50.0,4,Volksparkstadion,57000,€-6.00m,2017,../data/raw/transfermarkt-scraper/2017/clubs.j...,https://www.transfermarkt.co.uk/hamburger-sv/s...,14700000.0,98225000.0,79
1,2321033,L1,2013,1. Matchday,2013-08-10,23.0,86.0,0.0,1.0,13.0,7.0,Torsten Lieberknecht,Robin Dutt,EINTRACHT-Stadion,23000.0,Deniz Aytekin,https://www.transfermarkt.co.uk/eintracht-brau...,4-3-2-1,4-3-1-2,Eintracht Braunschweig,Sportverein Werder Bremen von 1899,0:1,domestic_league,bundesliga,bundesliga,first_tier,domestic_league,40,Germany,L1,europa,https://www.transfermarkt.co.uk/bundesliga/sta...,True,23,eintracht-braunschweig,Eintracht Braunschweig,L1,30,25.6,11,36.7,2,EINTRACHT-Stadion,23325,+€750k,2013,../data/raw/transfermarkt-scraper/2013/clubs.j...,https://www.transfermarkt.co.uk/eintracht-brau...,4000000.0,20625000.0,13,86,sv-werder-bremen,Sportverein Werder Bremen von 1899,L1,27,26.6,12,44.4,6,Weserstadion,42100,€-1.13m,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/sv-werder-brem...,160500000.0,427750000.0,147
2,2321044,L1,2013,2. Matchday,2013-08-18,16.0,23.0,2.0,1.0,1.0,15.0,Jürgen Klopp,Torsten Lieberknecht,SIGNAL IDUNA PARK,80200.0,Peter Sippel,https://www.transfermarkt.co.uk/borussia-dortm...,4-2-3-1,4-3-2-1,Borussia Dortmund,Eintracht Braunschweig,2:1,domestic_league,bundesliga,bundesliga,first_tier,domestic_league,40,Germany,L1,europa,https://www.transfermarkt.co.uk/bundesliga/sta...,True,16,borussia-dortmund,Borussia Dortmund,L1,28,25.2,14,50.0,18,SIGNAL IDUNA PARK,81365,€-30.60m,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/borussia-dortm...,462925000.0,829200000.0,363,23,eintracht-braunschweig,Eintracht Braunschweig,L1,30,25.6,11,36.7,2,EINTRACHT-Stadion,23325,+€750k,2013,../data/raw/transfermarkt-scraper/2013/clubs.j...,https://www.transfermarkt.co.uk/eintracht-brau...,4000000.0,20625000.0,13
3,2321060,L1,2013,3. Matchday,2013-08-25,23.0,24.0,0.0,2.0,18.0,11.0,Torsten Lieberknecht,Armin Veh,EINTRACHT-Stadion,23325.0,Wolfgang Stark,https://www.transfermarkt.co.uk/eintracht-brau...,4-3-2-1,4-2-3-1,Eintracht Braunschweig,Eintracht Frankfurt Fußball AG,0:2,domestic_league,bundesliga,bundesliga,first_tier,domestic_league,40,Germany,L1,europa,ht

In [21]:
columns = [
    'game_id',
    'competition_id',
    'season',
    'round',
    'date',
    'home_club_id',
    'away_club_id',
    'home_club_goals',
    'away_club_goals',
    'home_club_position',
    'away_club_position',
    'home_club_manager_name',
    'away_club_manager_name',
    'stadium',
    'attendance',
    'referee',
    'home_club_formation',
    'away_club_formation',
    'name',
    'sub_type',
    'type',
    'country_id',
    'is_major_national_league',

    'squad_size_home',
    'average_age_home',
    'foreigners_number_home',
    'foreigners_percentage_home',
    'national_team_players_home',
    'stadium_name_home',
    'stadium_seats_home',
    'net_transfer_record_home',
    'last_season_home',
    'total_market_value_home',
    'total_market_value_max_home',
    'total_is_win_home',

    'squad_size_away',
    'average_age_away',
    'foreigners_number_away',
    'foreigners_percentage_away',
    'national_team_players_away',
    'stadium_name_away',
    'stadium_seats_away',
    'net_transfer_record_away',
    'last_season_away',
    'total_market_value_away',
    'total_market_value_max_away',
    'total_is_win_away'
]

In [22]:
df = df[columns]

In [23]:
print(df.shape)
df.info()

(58486, 47)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58486 entries, 0 to 58485
Data columns (total 47 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   game_id                      58486 non-null  int64  
 1   competition_id               58486 non-null  object 
 2   season                       58486 non-null  int64  
 3   round                        58486 non-null  object 
 4   date                         58486 non-null  object 
 5   home_club_id                 58486 non-null  float64
 6   away_club_id                 58486 non-null  float64
 7   home_club_goals              58486 non-null  float64
 8   away_club_goals              58486 non-null  float64
 9   home_club_position           51559 non-null  float64
 10  away_club_position           51559 non-null  float64
 11  home_club_manager_name       58308 non-null  object 
 12  away_club_manager_name       58308 non-null  object 
 13  stad

# Підготовка даних

In [24]:
df.head(10)

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,away_club_position,home_club_manager_name,away_club_manager_name,stadium,attendance,referee,home_club_formation,away_club_formation,name,sub_type,type,country_id,is_major_national_league,squad_size_home,average_age_home,foreigners_number_home,foreigners_percentage_home,national_team_players_home,stadium_name_home,stadium_seats_home,net_transfer_record_home,last_season_home,total_market_value_home,total_market_value_max_home,total_is_win_home,squad_size_away,average_age_away,foreigners_number_away,foreigners_percentage_away,national_team_players_away,stadium_name_away,stadium_seats_away,net_transfer_record_away,last_season_away,total_market_value_away,total_market_value_max_away,total_is_win_away
0,2321027,L1,2013,1. Matchday,2013-08-11,33.0,41.0,3.0,3.0,8.0,9.0,Jens Keller,Thorsten Fink,Veltins-Arena,61973.0,Manuel Gräfe,4-2-3-1,4-2-3-1,bundesliga,first_tier,domestic_league,40,True,27,26.4,12,44.4,4,Veltins-Arena,62271,+€10.30m,2022,26375000.0,190875000.0,167,30,23.6,15,50.0,4,Volksparkstadion,57000,€-6.00m,2017,14700000.0,98225000.0,79
1,2321033,L1,2013,1. Matchday,2013-08-10,23.0,86.0,0.0,1.0,13.0,7.0,Torsten Lieberknecht,Robin Dutt,EINTRACHT-Stadion,23000.0,Deniz Aytekin,4-3-2-1,4-3-1-2,bundesliga,first_tier,domestic_league,40,True,30,25.6,11,36.7,2,EINTRACHT-Stadion,23325,+€750k,2013,4000000.0,20625000.0,13,27,26.6,12,44.4,6,Weserstadion,42100,€-1.13m,2024,160500000.0,427750000.0,147
2,2321044,L1,2013,2. Matchday,2013-08-18,16.0,23.0,2.0,1.0,1.0,15.0,Jürgen Klopp,Torsten Lieberknecht,SIGNAL IDUNA PARK,80200.0,Peter Sippel,4-2-3-1,4-3-2-1,bundesliga,first_tier,domestic_league,40,True,28,25.2,14,50.0,18,SIGNAL IDUNA PARK,81365,€-30.60m,2024,462925000.0,829200000.0,363,30,25.6,11,36.7,2,EINTRACHT-Stadion,23325,+€750k,2013,4000000.0,20625000.0,13
3,2321060,L1,2013,3. Matchday,2013-08-25,23.0,24.0,0.0,2.0,18.0,11.0,Torsten Lieberknecht,Armin Veh,EINTRACHT-Stadion,23325.0,Wolfgang Stark,4-3-2-1,4-2-3-1,bundesliga,first_tier,domestic_league,40,True,30,25.6,11,36.7,2,EINTRACHT-Stadion,23325,+€750k,2013,4000000.0,20625000.0,13,27,25.4,20,74.1,10,Deutsche Bank Park,58000,+€48.19m,2024,386935000.0,748375000.0,237
4,2321072,L1,2013,5. Matchday,2013-09-14,16.0,41.0,6.0,2.0,1.0,15.0,Jürgen Klopp,Thorsten Fink,SIGNAL IDUNA PARK,80645.0,Tobias Welz,4-2-3-1,3-5-2,bundesliga,first_tier,domestic_league,40,True,28,25.2,14,50.0,18,SIGNAL IDUNA PARK,81365,€-30.60m,2024,462925000.0,829200000.0,363,30,23.6,15,50.0,4,Volksparkstadion,57000,€-6.00m,2017,14700000.0,98225000.0,79
5,2321079,L1,2013,5. Matchday,2013-09-15,23.0,4.0,1.0,1.0,18.0,16.0,Torsten Lieberknecht,Michael Wiesinger,EINTRACHT-Stadion,22570.0,Marco Fritz,4-4-2 double 6,4-2-3-1,bundesliga,first_tier,domestic_league,40,True,30,25.6,11,36.7,2,EINTRACHT-Stadion,23325,+€750k,2013,4000000.0,20625000.0,13,33,25.2,7,21.2,2,Max-Morlock-Stadion,50000,+€2.63m,2018,12300000.0,80725000.0,34
6,2321086,L1,2013,6. Matchday,2013-09-21,4.0,16.0,1.0,1.0,15.0,1.0,Michael Wiesinger,Jürgen Klopp,Max-Morlock-Stadion,50000.0,Knut Kircher,4-2-3-1,4-2-3-1,bundesliga,first_tier,domestic_league,40,True,33,25.2,7,21.2,2,Max-Morlock-Stadion,50000,+€2.63m,2018,12300000.0,80725000.0,34,28,25.2,14,50.0,18,SIGNAL IDUNA PARK,81365,€-30.60m,2024,462925000.0,829200000.0,363
7,2321152,L1,2013,11. Matchday,2013-11-02,44.0,33.0,0.0,2.0,7.0,6.0,Jos Luhukay,Jens Keller,Olympiastadion Berlin,69277.0,Günter Perl,4-2-3-1,4-2-3-1,bundesliga,first_tier,domestic_league,40,True,36,24.1,14,38.9,7,Olympiastadion Berlin,74667,+€6.70m,2022,35235000.0,241000000.0,127,27,26.4,12,44.4,4,Veltins-Arena,62271,+€10.30m,2022,26375000.0,190875000.0,167
8,2321205,L1,2013,17. Matchday,2013-12-21,41.0,39.0,2.0,3.0,14.0,9.0,Bert van Marwijk,Thomas Tuchel,Volksparkstadion,50000.0,Bastian Dankert,4-2-3-1,4-4-2 Diamond,bundesliga,first_tier,domestic_league,40,True,30,23.6,15,50.0,4,Volksparkstadion,57000,€-6.00m,2017,14700000.0,98225

### Обробка пропущених значень

In [25]:
null_counts = df.isnull().sum()
null_counts = null_counts[null_counts != 0]
null_counts

home_club_position             6927
away_club_position             6927
home_club_manager_name          178
away_club_manager_name          178
stadium                          52
attendance                     6658
referee                         172
home_club_formation            5007
away_club_formation            5010
average_age_home               2365
foreigners_percentage_home     3047
total_market_value_home          54
total_market_value_max_home      54
average_age_away               2380
foreigners_percentage_away     3056
total_market_value_away          55
total_market_value_max_away      55
dtype: int64

In [26]:
null_counts = null_counts.index.tolist()

In [27]:
null_counts = df[null_counts]
null_counts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58486 entries, 0 to 58485
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   home_club_position           51559 non-null  float64
 1   away_club_position           51559 non-null  float64
 2   home_club_manager_name       58308 non-null  object 
 3   away_club_manager_name       58308 non-null  object 
 4   stadium                      58434 non-null  object 
 5   attendance                   51828 non-null  float64
 6   referee                      58314 non-null  object 
 7   home_club_formation          53479 non-null  object 
 8   away_club_formation          53476 non-null  object 
 9   average_age_home             56121 non-null  float64
 10  foreigners_percentage_home   55439 non-null  float64
 11  total_market_value_home      58432 non-null  float64
 12  total_market_value_max_home  58432 non-null  float64
 13  average_age_away

In [28]:
from numpy import dtype

float_cols = [col for col in null_counts.columns if null_counts[col].dtype == dtype('float64')]
float_cols

['home_club_position',
 'away_club_position',
 'attendance',
 'average_age_home',
 'foreigners_percentage_home',
 'total_market_value_home',
 'total_market_value_max_home',
 'average_age_away',
 'foreigners_percentage_away',
 'total_market_value_away',
 'total_market_value_max_away']

In [29]:
object_cols = [col for col in null_counts.columns if null_counts[col].dtype == dtype('O')]
object_cols

['home_club_manager_name',
 'away_club_manager_name',
 'stadium',
 'referee',
 'home_club_formation',
 'away_club_formation']

In [30]:
# Заповнити числовим значенням (наприклад, середнім, медіаною, нулем)
for col in float_cols:
    df[col] = df[col].fillna(df[col].median())

In [31]:
for col in object_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

In [32]:
del float_cols, object_cols

### Обробка дублікатів

In [33]:
print("Кількість дубльованих рядків (до видалення):", df.duplicated().sum())

Кількість дубльованих рядків (до видалення): 0


### Перетворення категоріальних даних в числові

In [34]:
df.head(10)

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,away_club_position,home_club_manager_name,away_club_manager_name,stadium,attendance,referee,home_club_formation,away_club_formation,name,sub_type,type,country_id,is_major_national_league,squad_size_home,average_age_home,foreigners_number_home,foreigners_percentage_home,national_team_players_home,stadium_name_home,stadium_seats_home,net_transfer_record_home,last_season_home,total_market_value_home,total_market_value_max_home,total_is_win_home,squad_size_away,average_age_away,foreigners_number_away,foreigners_percentage_away,national_team_players_away,stadium_name_away,stadium_seats_away,net_transfer_record_away,last_season_away,total_market_value_away,total_market_value_max_away,total_is_win_away
0,2321027,L1,2013,1. Matchday,2013-08-11,33.0,41.0,3.0,3.0,8.0,9.0,Jens Keller,Thorsten Fink,Veltins-Arena,61973.0,Manuel Gräfe,4-2-3-1,4-2-3-1,bundesliga,first_tier,domestic_league,40,True,27,26.4,12,44.4,4,Veltins-Arena,62271,+€10.30m,2022,26375000.0,190875000.0,167,30,23.6,15,50.0,4,Volksparkstadion,57000,€-6.00m,2017,14700000.0,98225000.0,79
1,2321033,L1,2013,1. Matchday,2013-08-10,23.0,86.0,0.0,1.0,13.0,7.0,Torsten Lieberknecht,Robin Dutt,EINTRACHT-Stadion,23000.0,Deniz Aytekin,4-3-2-1,4-3-1-2,bundesliga,first_tier,domestic_league,40,True,30,25.6,11,36.7,2,EINTRACHT-Stadion,23325,+€750k,2013,4000000.0,20625000.0,13,27,26.6,12,44.4,6,Weserstadion,42100,€-1.13m,2024,160500000.0,427750000.0,147
2,2321044,L1,2013,2. Matchday,2013-08-18,16.0,23.0,2.0,1.0,1.0,15.0,Jürgen Klopp,Torsten Lieberknecht,SIGNAL IDUNA PARK,80200.0,Peter Sippel,4-2-3-1,4-3-2-1,bundesliga,first_tier,domestic_league,40,True,28,25.2,14,50.0,18,SIGNAL IDUNA PARK,81365,€-30.60m,2024,462925000.0,829200000.0,363,30,25.6,11,36.7,2,EINTRACHT-Stadion,23325,+€750k,2013,4000000.0,20625000.0,13
3,2321060,L1,2013,3. Matchday,2013-08-25,23.0,24.0,0.0,2.0,18.0,11.0,Torsten Lieberknecht,Armin Veh,EINTRACHT-Stadion,23325.0,Wolfgang Stark,4-3-2-1,4-2-3-1,bundesliga,first_tier,domestic_league,40,True,30,25.6,11,36.7,2,EINTRACHT-Stadion,23325,+€750k,2013,4000000.0,20625000.0,13,27,25.4,20,74.1,10,Deutsche Bank Park,58000,+€48.19m,2024,386935000.0,748375000.0,237
4,2321072,L1,2013,5. Matchday,2013-09-14,16.0,41.0,6.0,2.0,1.0,15.0,Jürgen Klopp,Thorsten Fink,SIGNAL IDUNA PARK,80645.0,Tobias Welz,4-2-3-1,3-5-2,bundesliga,first_tier,domestic_league,40,True,28,25.2,14,50.0,18,SIGNAL IDUNA PARK,81365,€-30.60m,2024,462925000.0,829200000.0,363,30,23.6,15,50.0,4,Volksparkstadion,57000,€-6.00m,2017,14700000.0,98225000.0,79
5,2321079,L1,2013,5. Matchday,2013-09-15,23.0,4.0,1.0,1.0,18.0,16.0,Torsten Lieberknecht,Michael Wiesinger,EINTRACHT-Stadion,22570.0,Marco Fritz,4-4-2 double 6,4-2-3-1,bundesliga,first_tier,domestic_league,40,True,30,25.6,11,36.7,2,EINTRACHT-Stadion,23325,+€750k,2013,4000000.0,20625000.0,13,33,25.2,7,21.2,2,Max-Morlock-Stadion,50000,+€2.63m,2018,12300000.0,80725000.0,34
6,2321086,L1,2013,6. Matchday,2013-09-21,4.0,16.0,1.0,1.0,15.0,1.0,Michael Wiesinger,Jürgen Klopp,Max-Morlock-Stadion,50000.0,Knut Kircher,4-2-3-1,4-2-3-1,bundesliga,first_tier,domestic_league,40,True,33,25.2,7,21.2,2,Max-Morlock-Stadion,50000,+€2.63m,2018,12300000.0,80725000.0,34,28,25.2,14,50.0,18,SIGNAL IDUNA PARK,81365,€-30.60m,2024,462925000.0,829200000.0,363
7,2321152,L1,2013,11. Matchday,2013-11-02,44.0,33.0,0.0,2.0,7.0,6.0,Jos Luhukay,Jens Keller,Olympiastadion Berlin,69277.0,Günter Perl,4-2-3-1,4-2-3-1,bundesliga,first_tier,domestic_league,40,True,36,24.1,14,38.9,7,Olympiastadion Berlin,74667,+€6.70m,2022,35235000.0,241000000.0,127,27,26.4,12,44.4,4,Veltins-Arena,62271,+€10.30m,2022,26375000.0,190875000.0,167
8,2321205,L1,2013,17. Matchday,2013-12-21,41.0,39.0,2.0,3.0,14.0,9.0,Bert van Marwijk,Thomas Tuchel,Volksparkstadion,50000.0,Bastian Dankert,4-2-3-1,4-4-2 Diamond,bundesliga,first_tier,domestic_league,40,True,30,23.6,15,50.0,4,Volksparkstadion,57000,€-6.00m,2017,14700000.0,98225

In [35]:
object_cols = [col for col in df.columns if df[col].dtype == dtype('O')]
object_cols

['competition_id',
 'round',
 'date',
 'home_club_manager_name',
 'away_club_manager_name',
 'stadium',
 'referee',
 'home_club_formation',
 'away_club_formation',
 'name',
 'sub_type',
 'type',
 'stadium_name_home',
 'net_transfer_record_home',
 'stadium_name_away',
 'net_transfer_record_away']

In [36]:
df.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64'), dtype('bool')],
      dtype=object)

In [37]:
# Обробка дати
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['dayofweek'] = df['date'].dt.dayofweek  # 0 - понеділок, 6 - неділя

In [38]:
# Щоб закодувати home_club_formation і away_club_formation однаково (тобто, щоб однакові значення в обох колонках отримали однаковий код), потрібно:
# 1. Зібрати всі унікальні значення з обох колонок
unique_values = pd.unique(
    pd.concat([
        df['home_club_formation'].astype(str),
        df['away_club_formation'].astype(str)
    ])
)

# 2. Створити mapping
value_to_code = {val: i for i, val in enumerate(unique_values)}

# 3. Замапити коди на обидві колонки
df['home_club_formation_code'] = df['home_club_formation'].astype(str).map(value_to_code)
df['away_club_formation_code'] = df['away_club_formation'].astype(str).map(value_to_code)

In [39]:
joblib.dump(value_to_code, '../../neural_networks/v0/res/transformers/formation_code_map.joblib')

['../../neural_networks/v0/res/transformers/formation_code_map.joblib']

In [40]:
print((df[df['home_club_formation_code'] == df['away_club_formation_code']]['home_club_formation'] == df[df['home_club_formation_code'] == df['away_club_formation_code']]['away_club_formation']).all())
df[df['home_club_formation_code'] == df['away_club_formation_code']][['home_club_formation', 'away_club_formation']]

True

,home_club_formation,away_club_formation
0,4-2-3-1,4-2-3-1
6,4-2-3-1,4-2-3-1
7,4-2-3-1,4-2-3-1
10,4-2-3-1,4-2-3-1
13,4-2-3-1,4-2-3-1
...,...,...
58476,4-2-3-1,4-2-3-1
58477,4-2-3-1,4-2-3-1
58480,4-3-3 Defending,4-3-3 Defending
58483,4-3-3 Defending,4-3-3 Defending


In [41]:
def parse_transfer_value(value):
    if pd.isnull(value):
        return None
    value = value.replace('€', '').replace('+', '').replace(' ', '')
    sign = -1 if '-' in value else 1
    value = value.replace('-', '')
    if value.endswith('m'):
        num = float(value[:-1]) * 1_000_000
    elif value.endswith('k'):
        num = float(value[:-1]) * 1_000
    else:
        num = float(value)
    return sign * num

# Приклад для DataFrame:
df['net_transfer_record_home_num'] = df['net_transfer_record_home'].apply(parse_transfer_value)
df['net_transfer_record_away_num'] = df['net_transfer_record_away'].apply(parse_transfer_value)

In [42]:
df[[
    'net_transfer_record_home', 'net_transfer_record_home_num',
    'net_transfer_record_away', 'net_transfer_record_away_num'
]]

,net_transfer_record_home,net_transfer_record_home_num,net_transfer_record_away,net_transfer_record_away_num
0,+€10.30m,10300000.0,€-6.00m,-6000000.0
1,+€750k,750000.0,€-1.13m,-1130000.0
2,€-30.60m,-30600000.0,+€750k,750000.0
3,+€750k,750000.0,+€48.19m,48190000.0
4,€-30.60m,-30600000.0,€-6.00m,-6000000.0
...,...,...,...,...
58481,€-900k,-900000.0,+-0,-0.0
58482,+-0,-0.0,€-900k,-900000.0
58483,€-900k,-900000.0,+-0,-0.0
58484,+-0,-0.0,€-900k,-900000.0


In [43]:
# Щоб закодувати всі стадіони однаково (тобто, щоб однакові значення в обох колонках отримали однаковий код), потрібно:
# 1. Зібрати всі унікальні значення з обох колонок
unique_values = pd.unique(
    pd.concat([
        df['stadium'].astype(str),
        df['stadium_name_home'].astype(str),
        df['stadium_name_away'].astype(str)
    ])
)

# 2. Створити mapping
value_to_code = {val: i for i, val in enumerate(unique_values)}

# 3. Замапити коди на обидві колонки
df['stadium_code'] = df['stadium'].astype(str).map(value_to_code)
df['stadium_name_home_code'] = df['stadium_name_home'].astype(str).map(value_to_code)
df['stadium_name_away_code'] = df['stadium_name_away'].astype(str).map(value_to_code)

In [44]:
joblib.dump(value_to_code, '../../neural_networks/v0/res/transformers/stadium_code_map.joblib')

['../../neural_networks/v0/res/transformers/stadium_code_map.joblib']

In [45]:
label_encoding = [
    'competition_id',
    'round',
    # 'date',
    'home_club_manager_name',
    'away_club_manager_name',
    # 'stadium',
    'referee',
    # 'home_club_formation',
    # 'away_club_formation',
    'name',
    'sub_type',
    'type',
    # 'stadium_name_home',
    # 'net_transfer_record_home',
    # 'stadium_name_away',
    # 'net_transfer_record_away'
]

In [46]:
label_encoding_maps = {}

for col in label_encoding:
    codes, uniques = pd.factorize(df[col])
    df[col] = codes
    label_encoding_maps[col] = list(uniques)

# Зберегти мапи
joblib.dump(label_encoding_maps, '../../neural_networks/v0/res/transformers/label_encoding_maps.joblib')

['../../neural_networks/v0/res/transformers/label_encoding_maps.joblib']

### Обробка викидів

In [47]:
Q1 = df['home_club_goals'].quantile(0.25)
Q3 = df['home_club_goals'].quantile(0.75)
IQR = Q3 - Q1
df[(df['home_club_goals'] >= Q1 - 1.5 * IQR) & (df['home_club_goals'] <= Q3 + 1.5 * IQR)].shape
df.shape

(58486, 58)

### Обробка bool колонок

In [48]:
df['is_major_national_league'] = df['is_major_national_league'].astype(float)

### Нормалізація

In [49]:
df.head()

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,away_club_position,home_club_manager_name,away_club_manager_name,stadium,attendance,referee,home_club_formation,away_club_formation,name,sub_type,type,country_id,is_major_national_league,squad_size_home,average_age_home,foreigners_number_home,foreigners_percentage_home,national_team_players_home,stadium_name_home,stadium_seats_home,net_transfer_record_home,last_season_home,total_market_value_home,total_market_value_max_home,total_is_win_home,squad_size_away,average_age_away,foreigners_number_away,foreigners_percentage_away,national_team_players_away,stadium_name_away,stadium_seats_away,net_transfer_record_away,last_season_away,total_market_value_away,total_market_value_max_away,total_is_win_away,year,month,day,dayofweek,home_club_formation_code,away_club_formation_code,net_transfer_record_home_num,net_transfer_record_away_num,stadium_code,stadium_name_home_code,stadium_name_away_code
0,2321027,0,2013,0,2013-08-11,33.0,41.0,3.0,3.0,8.0,9.0,0,0,Veltins-Arena,61973.0,0,4-2-3-1,4-2-3-1,0,0,0,40,1.0,27,26.4,12,44.4,4,Veltins-Arena,62271,+€10.30m,2022,26375000.0,190875000.0,167,30,23.6,15,50.0,4,Volksparkstadion,57000,€-6.00m,2017,14700000.0,98225000.0,79,2013,8,11,6,0,0,10300000.0,-6000000.0,0,0,5
1,2321033,0,2013,0,2013-08-10,23.0,86.0,0.0,1.0,13.0,7.0,1,1,EINTRACHT-Stadion,23000.0,1,4-3-2-1,4-3-1-2,0,0,0,40,1.0,30,25.6,11,36.7,2,EINTRACHT-Stadion,23325,+€750k,2013,4000000.0,20625000.0,13,27,26.6,12,44.4,6,Weserstadion,42100,€-1.13m,2024,160500000.0,427750000.0,147,2013,8,10,5,1,7,750000.0,-1130000.0,1,1,115
2,2321044,0,2013,1,2013-08-18,16.0,23.0,2.0,1.0,1.0,15.0,2,2,SIGNAL IDUNA PARK,80200.0,2,4-2-3-1,4-3-2-1,0,0,0,40,1.0,28,25.2,14,50.0,18,SIGNAL IDUNA PARK,81365,€-30.60m,2024,462925000.0,829200000.0,363,30,25.6,11,36.7,2,EINTRACHT-Stadion,23325,+€750k,2013,4000000.0,20625000.0,13,2013,8,18,6,0,1,-30600000.0,750000.0,2,2,1
3,2321060,0,2013,2,2013-08-25,23.0,24.0,0.0,2.0,18.0,11.0,1,3,EINTRACHT-Stadion,23325.0,3,4-3-2-1,4-2-3-1,0,0,0,40,1.0,30,25.6,11,36.7,2,EINTRACHT-Stadion,23325,+€750k,2013,4000000.0,20625000.0,13,27,25.4,20,74.1,10,Deutsche Bank Park,58000,+€48.19m,2024,386935000.0,748375000.0,237,2013,8,25,6,1,0,750000.0,48190000.0,1,1,633
4,2321072,0,2013,3,2013-09-14,16.0,41.0,6.0,2.0,1.0,15.0,2,0,SIGNAL IDUNA PARK,80645.0,4,4-2-3-1,3-5-2,0,0,0,40,1.0,28,25.2,14,50.0,18,SIGNAL IDUNA PARK,81365,€-30.60m,2024,462925000.0,829200000.0,363,30,23.6,15,50.0,4,Volksparkstadion,57000,€-6.00m,2017,14700000.0,98225000.0,79,2013,9,14,5,0,17,-30600000.0,-6000000.0,2,2,5


In [50]:
cols_for_norm = [
    # 'game_id', 
    'competition_id', 
    'season', 
    'round', 
    # 'date', 
    'home_club_id',
    'away_club_id', 
    # 'home_club_goals', 
    # 'away_club_goals',
    'home_club_position', 
    'away_club_position', 
    'home_club_manager_name',
    'away_club_manager_name', 
    # 'stadium', 
    'attendance', 
    'referee',
    # 'home_club_formation', 
    # 'away_club_formation', 
    'name', 
    'sub_type',
    'type', 
    'country_id', 
    'is_major_national_league', 
    'squad_size_home',
    'average_age_home', 
    'foreigners_number_home',
    'foreigners_percentage_home', 
    'national_team_players_home',
    # 'stadium_name_home', 
    'stadium_seats_home', 
    # 'net_transfer_record_home',
    'last_season_home', 
    'total_market_value_home',
    'total_market_value_max_home',
    'total_is_win_home',
    'squad_size_away',
    'average_age_away', 
    'foreigners_number_away',
    'foreigners_percentage_away', 
    'national_team_players_away',
    # 'stadium_name_away', 
    'stadium_seats_away', 
    # 'net_transfer_record_away',
    'last_season_away', 
    'total_market_value_away',
    'total_market_value_max_away',
    'total_is_win_away',
    'year', 
    'month', 
    'day',
    'dayofweek', 
    'home_club_formation_code', 
    'away_club_formation_code',
    'net_transfer_record_home_num', 
    'net_transfer_record_away_num',
    'stadium_code', 
    'stadium_name_home_code', 
    'stadium_name_away_code'
]

In [51]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[cols_for_norm] = scaler.fit_transform(df[cols_for_norm])

# Зберегти scaler
joblib.dump(scaler, '../../neural_networks/v0/res/transformers/scaler.joblib')

['../../neural_networks/v0/res/transformers/scaler.joblib']

In [52]:
scaler_y = StandardScaler()
df_scaled[['home_club_goals', 'away_club_goals']] = scaler_y.fit_transform(df_scaled[['home_club_goals', 'away_club_goals']])

# Зберегти scaler_y
joblib.dump(scaler_y, '../../neural_networks/v0/res/transformers/scaler_y.joblib')

['../../neural_networks/v0/res/transformers/scaler_y.joblib']

In [53]:
df_scaled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58486 entries, 0 to 58485
Data columns (total 58 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   game_id                       58486 non-null  int64         
 1   competition_id                58486 non-null  float64       
 2   season                        58486 non-null  float64       
 3   round                         58486 non-null  float64       
 4   date                          58486 non-null  datetime64[ns]
 5   home_club_id                  58486 non-null  float64       
 6   away_club_id                  58486 non-null  float64       
 7   home_club_goals               58486 non-null  float64       
 8   away_club_goals               58486 non-null  float64       
 9   home_club_position            58486 non-null  float64       
 10  away_club_position            58486 non-null  float64       
 11  home_club_manager_name      

In [54]:
df_scaled.shape

(58486, 58)

In [55]:
df_scaled.head()

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,away_club_position,home_club_manager_name,away_club_manager_name,stadium,attendance,referee,home_club_formation,away_club_formation,name,sub_type,type,country_id,is_major_national_league,squad_size_home,average_age_home,foreigners_number_home,foreigners_percentage_home,national_team_players_home,stadium_name_home,stadium_seats_home,net_transfer_record_home,last_season_home,total_market_value_home,total_market_value_max_home,total_is_win_home,squad_size_away,average_age_away,foreigners_number_away,foreigners_percentage_away,national_team_players_away,stadium_name_away,stadium_seats_away,net_transfer_record_away,last_season_away,total_market_value_away,total_market_value_max_away,total_is_win_away,year,month,day,dayofweek,home_club_formation_code,away_club_formation_code,net_transfer_record_home_num,net_transfer_record_away_num,stadium_code,stadium_name_home_code,stadium_name_away_code
0,2321027,-1.270845,-1.315944,-1.281584,2013-08-11,-0.374854,-0.374635,1.073348,1.456098,-0.249495,-0.080008,-1.294868,-1.261281,Veltins-Arena,2.403578,-1.416628,4-2-3-1,4-2-3-1,-1.190309,-0.271567,-0.339809,-1.081613,1.235075,0.198251,0.305435,-0.214818,-0.413885,-0.541103,Veltins-Arena,1.652369,+€10.30m,-0.346854,-0.652305,-0.534401,-0.089468,0.669524,-1.287581,0.278377,-0.105972,-0.538765,Volksparkstadion,1.384007,€-6.00m,-2.380332,-0.693390,-0.726491,-0.823074,-1.432451,0.356065,-0.544828,0.859005,-0.786416,-0.811003,0.374295,-0.086558,-1.456036,-1.470120,-1.439102
1,2321033,-1.270845,-1.315944,-1.281584,2013-08-10,-0.376031,-0.369285,-1.161622,-0.191235,0.755516,-0.480396,-1.292192,-1.258618,EINTRACHT-Stadion,0.169091,-1.411576,4-3-2-1,4-3-1-2,-1.190309,-0.271567,-0.339809,-1.081613,1.235075,0.670695,-0.148303,-0.378781,-0.834506,-0.901071,EINTRACHT-Stadion,-0.358401,+€750k,-4.002445,-0.734329,-0.889533,-1.374058,0.197681,0.420807,-0.213135,-0.411861,-0.178044,Weserstadion,0.613877,€-1.13m,0.466240,-0.157395,-0.036848,-0.254463,-1.432451,0.356065,-0.656934,0.293306,-0.629760,0.275239,0.105489,0.050652,-1.448966,-1.463678,-0.730257
2,2321044,-1.270845,-1.315944,-1.225685,2013-08-18,-0.376855,-0.376775,0.328358,-0.191235,-1.656509,1.121156,-1.289517,-1.255955,SIGNAL IDUNA PARK,3.448608,-1.406524,4-2-3-1,4-3-2-1,-1.190309,-0.271567,-0.339809,-1.081613,1.235075,0.355732,-0.375172,0.113108,-0.107979,1.978673,SIGNAL IDUNA PARK,2.638187,€-30.60m,0.465500,0.948022,0.797112,1.545465,0.669524,-0.148656,-0.376972,-0.832459,-0.899485,EINTRACHT-Stadion,-0.356539,+€750k,-4.006945,-0.732725,-0.888896,-1.374962,-1.432451,0.356065,0.239909,0.859005,-0.786416,-0.655825,-0.776929,0.103620,-1.441897,-1.457235,-1.464878
3,2321060,-1.270845,-1.315944,-1.169787,2013-08-25,-0.376031,-0.376657,-1.161622,0.632432,1.760526,0.320380,-1.292192,-1.253292,EINTRACHT-Stadion,0.187725,-1.401473,4-3-2-1,4-2-3-1,-1.190309,-0.271567,-0.339809,-1.081613,1.235075,0.670695,-0.148303,-0.378781,-0.834506,-0.901071,EINTRACHT-Stadion,-0.358401,+€750k,-4.002445,-0.734329,-0.889533,-1.374058,0.197681,-0.262548,1.097563,1.210444,0.543397,Deutsche Bank Park,1.435693,+€48.19m,0.466240,0.675032,0.634169,0.498112,-1.432451,0.356065,1.024647,0.859005,-0.629760,-0.811003,0.105489,1.440217,-1.448966,-1.463678,2.607757
4,2321072,-1.270845,-1.315944,-1.113888,2013-09-14,-0.376855,-0.374635,3.308319,0.632432,-1.656509,1.121156,-1.289517,-1.261281,SIGNAL IDUNA PARK,3.474122,-1.396421,4-2-3-1,3-5-2,-1.190309,-0.271567,-0.339809,-1.081613,1.235075,0.355732,-0.375172,0.113108,-0.107979,1.978673,SIGNAL IDUNA PARK,2.638187,€-30.60m,0.465500,0.948022,0.797112,1.545465,0.669524,-1.287581,0.278377,-0.105972,-0.538765,Volksparkstadion,1.384007,€-6.00m,-2.380332,-0.693390,-0.726491,-0.823074,-1.432451,0.619340,-0.208512,0.293306,-0.786416,1.827013,-0.776929,-0.086558,-1.441897,-1.457235,-1.439102


# Збереження

In [56]:
df_scaled.drop([
    'game_id', 'date', 'net_transfer_record_away',
    'stadium', 'home_club_formation', 'away_club_formation',
    'stadium_name_home', 'net_transfer_record_home', 'stadium_name_away',
], inplace=True, axis=1)

In [57]:
df_scaled.to_csv('../processed/prepared_data.csv', index=False)

# Створення даних для frontend

In [68]:
for_frontend = '../for_frontend'
import json

In [69]:
with open(for_frontend + '/clubs_name.json', 'w') as f:
    json.dump(list(clubs['name']), f)

In [70]:
with open(for_frontend + '/referees.json', 'w') as f:
    json.dump(list(games['referee']), f)

In [71]:
with open(for_frontend + '/stadiums.json', 'w') as f:
    json.dump(list(games['stadium']), f)

In [72]:
with open(for_frontend + '/competitions_code.json', 'w') as f:
    json.dump(list(competitions['competition_code']), f)